In [1]:
import pandas as pd
import numpy as np
import config
from sqlalchemy import text
print(config.version)
print(config.connection_string)
pd.set_option('display.max_columns', None)

3.2.0.Dev
postgresql://postgres:sa@localhost/icip


In [2]:
specialite=10
year=21
dept=6
source="PS"
time=30
# accessibilite_exp=-0.12 #0.08 pour 45
accessibilite_exp=-(75-time)*4/1500
print(year, specialite, source, time, accessibilite_exp)

21 10 PS 30 -0.12


In [3]:
d="an.dept_id" if dept is None else str(dept) # Non utilisé
sources: dict[str, str] = {
    "PS": f"""
select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and tds.date_source_id >= {year}00 and  tds.date_source_id < {year+1}00
and ar.dept_id={dept}
group by ps.id, c.id, an.id, i.id
""",
    "PA": f"""
select pa.id, pa.nom, pa.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne_activite pa
join personne_activite_diplome pad on pad.personne_activite_id=pa.id
join diplome d on pad.diplome_id=d.id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join pa_adresse_norm_date_source pands on pands.personne_activite_id = pa.id
join adresse_norm an on pands.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and pa.code_mode_exercice='L'
and pands.date_source_id/100={year}
and an.dept_id={dept}
group by pa.id, an.id, i.id
""",
    "RPPS": f"""
select personne.id, personne.nom, personne.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from personne
join diplome_obtenu dio on dio.personne_id = personne.id
join diplome d on d.id=dio.diplome_id
join profession_diplome pd on pd.diplome_id=d.id
join profession p on pd.profession_id=p.id
join specialite_profession sp on sp.profession_id=p.id
join coord c on c.personne_id=personne.id
join adresse_norm an on c.adresse_norm_id=an.id
join iris.iris i on i.code=an.iris
where sp.specialite_id={specialite}
and personne.date_maj > '20{year}-01-01' and date_effet < '20{year}-01-01'
and an.dept_id={dept}
group by personne.id, an.id, i.id
"""}

sql = sources[source]
print(sql)


select ps.id, ps.nom, ps.prenom, an.dept_id, an.id adresse_norm_id, i.id iris, an.lon, an.lat from ps
join tarif t on t.ps_id = ps.id
join tarif_date_source tds on tds.tarif_id=t.id 
join cabinet c on t.cabinet_id=c.id
join adresse_raw ar on c.adresse_raw_id=ar.id
join adresse_norm an on ar.adresse_norm_id=an.id
join specialite_profession sp on sp.profession_id=t.profession_id
join iris.iris i on i.code=an.iris
where sp.specialite_id=10
and tds.date_source_id >= 2100 and  tds.date_source_id < 2200
and ar.dept_id=6
group by ps.id, c.id, an.id, i.id



In [4]:
df = pd.read_sql(sql, config.connection_string)
df

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604
...,...,...,...,...,...,...,...,...
1280,155689,SAROUX,DELPHINE,6,2305,60882401,7.258125,43.695605
1281,155690,PERADOTTO,CYRIL,6,2866,60290121,6.988390,43.550361
1282,155692,DONADIO,MANON,6,77494,61490102,7.313438,43.743446
1283,155827,BOUCHET BADEL,MARIE,6,2328,60690102,6.914032,43.646318


In [5]:
nb_ps = df["id"].nunique()
unique_ps = df.groupby(["id", "lon", "lat"])
print(f"Nb PS {nb_ps}")
print(f"Nb unique PS {len(unique_ps)}")

Nb PS 1203
Nb unique PS 1273


In [6]:
df["key"] = df["id"].astype(str) + "_" + df["lat"].astype(str) + "_" + df["lon"].astype(str)
df["nb_cabinet"] = df.groupby("id")["key"].transform("nunique")
df["nb_cabinet"].value_counts(normalize=True)

nb_cabinet
1    0.893385
2    0.094163
3    0.009339
4    0.003113
Name: proportion, dtype: float64

In [7]:
df["weight"] = 1 / df["nb_cabinet"]
df["nb"] = df.groupby("iris")["weight"].transform("sum")
df.head(10)

,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
0,16126,LEONARDI,DOMINIQUE,6,1415,60881903,7.255881,43.709653,16126_43.709653_7.255881,1,1.0,5.000000
1,16130,BENSUSSAN,JEAN,6,92990,60882801,7.227639,43.676026,16130_43.676026_7.227639,1,1.0,8.000000
2,16131,LARA,JEANNINE,6,92963,60880202,7.282865,43.695573,16131_43.695573_7.282865,1,1.0,3.000000
3,16132,GUEGAN,JEAN CLAUDE,6,1425,60270102,7.145627,43.657352,16132_43.657352_7.145627,1,1.0,5.000000
4,16148,CROQUET,PATRICE,6,1433,60690109,6.922279,43.651604,16148_43.651604_6.922279,1,1.0,10.500000
5,16151,DESGATS,MARC,6,1436,60300107,7.005844,43.579045,16151_43.579045_7.005844,1,1.0,5.000000
6,16153,DUBOIS,MICHEL,6,1438,61360000,7.449651,43.877046,16153_43.877046_7.449651,1,1.0,4.000000
7,16159,GIDEL,PIERRE JEAN,6,1466,60882601,7.242220,43.689322,16159_43.689322_7.24222,1,1.0,11.666667
8,16160,MASCHINO,JEAN MARC,6,1467,60290102,7.037066,43.546353,16160_43.546353_7.037066,1,1.0,3.000000
9,16161,SARDOU,YVES,6,1468,60830205,7.492397,43.791639,16161_43.791639_7.492397,1,1.0,1.500000


In [8]:
df = df.sort_values(by='iris')
df = df.drop_duplicates(subset=['iris', 'nb'])
df


,id,nom,prenom,dept_id,adresse_norm_id,iris,lon,lat,key,nb_cabinet,weight,nb
123,16425,DAEMERS,YVES,6,92989,60040102,7.072160,43.596934,16425_43.596934_7.07216,1,1.0,2.000000
904,113924,DIAMANT,GUY,6,77467,60040103,7.137517,43.567521,113924_43.567521_7.137517,1,1.0,1.000000
109,16404,MANDRILE,DIDIER,6,1651,60040104,7.092388,43.604533,16404_43.604533_7.092388,1,1.0,1.000000
31,16222,NICASTRO,SERGE,6,92975,60040105,7.117538,43.588105,16222_43.588105_7.117538,1,1.0,2.000000
1179,144024,FRICKEY,SARAH,6,2734,60040106,7.112369,43.606074,144024_43.606074_7.112369,1,1.0,8.666667
...,...,...,...,...,...,...,...,...,...,...,...,...
607,17388,ROUSSEAU,FRANCOISE,6,2635,61610103,7.123625,43.658047,17388_43.658047_7.123625,1,1.0,1.000000
1069,132333,AGOSTINI,PIERRE,6,81986,61610104,7.141591,43.641591,132333_43.641591_7.141591,2,0.5,3.000000
742,99028,BENASSAYAG,JEAN JACQUES,6,68080,61610105,7.140329,43.639824,99028_43.639824_7.140329,1,1.0,5.000000
301,16814,DI VINCENZO,DOMINIQUE,6,273951,61620000,7.612673,44.063665,16814_44.063665_7.612673,2,0.5,0.500000


In [9]:
def get_iris_matrix():
    sql = f"""
(select iris.id "iris1", iris.id "iris2", 0 "km", 0 "time", 0 "time_hp" from iris.iris
where id / 10000000 = {dept}
union
select iris_id_from as "iris1", iris_id_to "iris2", route_km "km", route_min "time", route_hp_min "time_hp" from iris.iris_matrix
where (iris_id_from/10000000={dept} or iris_id_to/10000000={dept})
and route_min <= {time}) order by "iris1", "iris2"
"""
    print(sql)
    return pd.read_sql(sql, config.connection_string)

# iris_matrix = get_iris_matrix()
iris_matrix = pd.read_csv("od_06_sql_rename.csv")
iris_matrix["iris"] = iris_matrix["iris2"].astype("int64")
iris_matrix
    

,iris1,iris2,km,time,iris
0,40080000,60530000,17,22,60530000
1,40080000,60990000,21,24,60990000
2,40390000,61540000,24,30,61540000
3,40420000,60530000,11,19,60530000
4,40420000,60710000,22,30,60710000
...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103
117765,831380000,60950101,18,19,60950101
117766,831380000,60950102,19,22,60950102
117767,831380000,60950103,18,20,60950103


In [10]:
def get_pop_iris():
    yy=min(21, year)
    sql = f"""
select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '{dept:02}%'
and year={yy}
"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

pop_iris = get_pop_iris()
pop_iris


select i.id iris, pi.iris iris_string, c.code code_commune, i.type type_iris, pi.pop0002, pi.pop0305, pi.pop0610, pi.pop1117, pi.pop1824, pi.pop2539, pi.pop4054, pi.pop5564, pi.pop6579, pi.pop80p
from iris.pop_iris pi
join iris.iris i on  pi.iris=i.code
join iris.commune c on i.commune_id=c.id
where com like '06%'
and year=21



,iris,iris_string,code_commune,type_iris,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,61230108,061230108,06123,A,88.787020,75.776145,135.840215,156.862889,129.702563,508.721610,440.454939,303.481339,433.313157,239.522400
1,61530000,061530000,06153,C,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321
2,61450000,061450000,06145,C,0.965986,0.965986,8.693878,11.591837,9.659864,16.421769,34.775510,12.557823,34.775510,11.591837
3,60120105,060120105,06012,A,68.356436,55.185368,179.895476,266.832339,226.432893,571.248108,831.040803,391.639115,351.412985,194.786785
4,60500000,060500000,06050,C,14.188627,14.119226,40.517897,47.603916,17.493201,71.924387,145.187085,83.001180,65.334467,26.630014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,60881606,060881606,06088,A,56.982735,81.433187,104.491854,148.417994,461.133207,399.230815,389.919891,334.254784,416.753195,228.940710
496,60881701,060881701,06088,A,24.463401,43.859704,85.003020,126.388729,282.367384,250.163180,355.063504,193.244615,223.033528,124.721315
497,60881804,060881804,06088,A,85.043040,83.274932,124.543590,225.112876,294.722303,673.422185,570.114686,285.987223,350.796724,200.210873
498,60881805,060881805,06088,A,65.721430,54.233925,76.225674,153.780470,257.227060,430.652161,329.042912,253.515770,331.504051,204.323195


In [11]:
apl = iris_matrix.merge(pop_iris, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
1,40080000,60990000,21,24,60990000,060990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272
2,40390000,61540000,24,30,61540000,061540000,06154,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582
4,40420000,60710000,22,30,60710000,060710000,06071,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,60850103,060850103,06085,A,57.428416,72.207652,135.459522,194.409719,141.793750,318.526897,479.908067,393.484524,562.460591,247.925725
117765,831380000,60950101,18,19,60950101,060950101,06095,A,77.910796,87.987431,210.729551,255.894450,155.595176,478.030563,685.139541,418.753123,589.352449,264.071324
117766,831380000,60950102,19,22,60950102,060950102,06095,A,67.975915,77.960010,155.903079,226.090400,145.568264,401.767992,529.572959,354.290116,427.490829,177.066411
117767,831380000,60950103,18,20,60950103,060950103,06095,A,46.976690,68.925850,125.728002,224.736092,103.186116,259.487677,537.700844,388.050677,528.083677,185.973993


In [12]:
accessibilite_fn = lambda x: np.exp(accessibilite_exp * x)

# apl["time"] = apl["time"].replace(0, 0.5)
apl["accessibilite_weight"] = accessibilite_fn(apl["time"])
apl.head(10)

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361
1,40080000,60990000,21,24,60990000,060990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
2,40390000,61540000,24,30,61540000,061540000,06154,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284
4,40420000,60710000,22,30,60710000,060710000,06071,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324
5,40420000,60990000,17,24,60990000,060990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135
6,40420000,61240000,15,30,61240000,061240000,06124,C,2.557377,0.852459,0.000000,0.000000,0.852459,6.819672,4.262295,7.672131,23.016393,5.967213,0.027324
7,40430000,60020000,15,20,60020000,060020000,06002,C,0.944444,0.000000,0.944444,0.000000,4.722222,2.833333,12.277778,12.277778,14.166667,2.833333,0.090718
8,40430000,60240000,8,11,60240000,060240000,06024,C,4.380952,5.476190,4.380952,16.428571,3.285714,19.714286,36.142857,20.809524,55.857143,17.523810,0.267135
9,40430000,60450000,18,23,60450000,060450000,06045,C,2.710965,0.903655,0.903655,1.807310,4.518275,11.840499,5.421930,17.169444,22.591374,8.132895,0.063292


In [13]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight
105136,61290000,60730000,14,17,60730000,060730000,06073,C,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029
105137,61290000,60800000,10,18,60800000,060800000,06080,C,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325
105138,61290000,60820000,23,30,60820000,060820000,06082,C,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324
105139,61290000,61020000,13,21,61020000,061020000,06102,C,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460
105140,61290000,61100000,12,19,61100000,061100000,06110,C,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284
105141,61290000,61110000,10,17,61110000,061110000,06111,C,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029
105142,61290000,61290000,0,0,61290000,061290000,06129,C,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000
105143,61290000,61460000,18,23,61460000,061460000,06146,C,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292
105144,61290000,61530000,14,21,61530000,061530000,06153,C,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460


In [14]:
cols = [col for col in apl.columns if "pop" in col]
cols

['pop0002',
 'pop0305',
 'pop0610',
 'pop1117',
 'pop1824',
 'pop2539',
 'pop4054',
 'pop5564',
 'pop6579',
 'pop80p']

In [15]:
sql = f"""
select o.* from apl.overrepresentation o
where o.year={year} and o.psp_spe_snds=1
"""
over = pd.read_sql(sql, config.connection_string)
# todo ajouter la colonne psp_spe_snds dans specialite
over

,year,psp_spe_snds,p0002,p0305,p0610,p1117,p1824,p2539,p4054,p5564,p6579,p80p
0,21,1,1.166949,0.539683,0.402415,0.420643,0.708842,0.929717,0.886061,1.074038,1.491386,2.405645


In [16]:
weights = over.values[0,2:]
weights

array([1.1669492 , 0.53968312, 0.40241483, 0.42064349, 0.70884153,
       0.92971694, 0.88606123, 1.07403771, 1.49138583, 2.40564513])

In [17]:
apl["pop_gp"] = sum(w * apl[c] for w, c in zip(weights, cols))
apl.sort_values(by='iris2')
apl.head(5)   

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,60530000,060530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.071361,167.589023
1,40080000,60990000,21,24,60990000,060990000,06099,C,44.960846,59.174287,115.477201,170.664745,99.908923,286.028889,372.666297,213.784254,309.368287,140.966272,0.056135,1899.726451
2,40390000,61540000,24,30,61540000,061540000,06154,C,13.220641,11.542128,38.449393,35.331972,16.699780,71.015373,85.106947,76.715979,113.173907,33.743879,0.027324,537.620953
3,40420000,60530000,11,19,60530000,060530000,06053,C,3.042553,3.042553,3.042553,15.212766,3.042553,19.269504,22.312057,20.283688,39.553191,14.198582,0.102284,167.589023
4,40420000,60710000,22,30,60710000,060710000,06071,C,12.947819,10.957307,29.872821,24.887047,20.940392,91.782711,104.799327,98.895928,130.928878,57.987769,0.027324,677.528167


In [18]:
apl[apl["iris1"]==61290000]

,iris1,iris2,km,time,iris,iris_string,code_commune,type_iris,pop0002,pop0305,pop0610,pop1117,pop1824,pop2539,pop4054,pop5564,pop6579,pop80p,accessibilite_weight,pop_gp
105136,61290000,60730000,14,17,60730000,060730000,06073,C,15.966553,14.947243,42.905088,55.876926,20.971505,96.981393,145.940458,117.149194,107.190565,35.071074,0.130029,671.865441
105137,61290000,60800000,10,18,60800000,060800000,06080,C,4.952830,1.981132,3.962264,3.962264,2.971698,10.896226,18.820755,16.839623,35.660377,4.952830,0.115325,122.207793
105138,61290000,60820000,23,30,60820000,060820000,06082,C,0.833333,0.833333,2.500000,6.666667,5.833333,13.333333,30.000000,18.333333,25.000000,11.666667,0.027324,133.386689
105139,61290000,61020000,13,21,61020000,061020000,06102,C,5.181818,1.727273,4.318182,10.363636,4.318182,12.954545,21.590909,31.954545,29.363636,11.227273,0.080460,152.433762
105140,61290000,61100000,12,19,61100000,061100000,06110,C,2.924370,2.924370,2.924370,6.823529,4.873950,11.697479,24.369748,35.092437,16.571429,7.798319,0.102284,126.126181
105141,61290000,61110000,10,17,61110000,061110000,06111,C,0.000000,0.000000,1.899160,3.798319,2.848739,15.193277,22.789916,26.588235,25.638655,14.243697,0.130029,139.759178
105142,61290000,61290000,0,0,61290000,061290000,06129,C,1.104023,4.449030,15.804142,40.071305,15.409834,45.894488,60.413865,35.273148,57.768530,24.811635,1.000000,317.755204
105143,61290000,61460000,18,23,61460000,061460000,06146,C,5.000000,3.000000,11.000000,16.000000,7.000000,23.000000,35.000000,28.000000,13.000000,8.000000,0.063292,144.674411
105144,61290000,61530000,14,21,61530000,061530000,06153,C,18.020482,23.026074,44.052137,67.079572,41.031287,120.144366,165.201707,152.189294,156.194759,48.060321,0.080460,878.583120


In [19]:
apl = apl.drop(["iris", "iris_string", "code_commune"], axis=1)
apl = apl.drop(cols, axis=1)
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp
0,40080000,60530000,17,22,C,0.071361,167.589023
1,40080000,60990000,21,24,C,0.056135,1899.726451
2,40390000,61540000,24,30,C,0.027324,537.620953
3,40420000,60530000,11,19,C,0.102284,167.589023
4,40420000,60710000,22,30,C,0.027324,677.528167


In [20]:
apl["iris"] = apl["iris1"]
apl.sort_values(by='iris2')
apl = apl.merge(df, on="iris", how="left", suffixes=('', ''))
apl

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb
0,40080000,60530000,17,22,C,0.071361,167.589023,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40080000,60990000,21,24,C,0.056135,1899.726451,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,40390000,61540000,24,30,C,0.027324,537.620953,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,40420000,60530000,11,19,C,0.102284,167.589023,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,40420000,60710000,22,30,C,0.027324,677.528167,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117764,831380000,60850103,34,29,A,0.030807,2922.034770,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117765,831380000,60950101,18,19,A,0.102284,3456.615760,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117766,831380000,60950102,19,22,A,0.071361,2669.222378,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117767,831380000,60950103,18,20,A,0.090718,2679.719687,831380000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
apl["nb"] = apl["nb"].fillna(0)
apl = apl.sort_values(by=['iris1', "iris2"])
apl["nb"].unique()

array([ 0.        ,  2.        ,  1.        ,  8.66666667,  4.        ,
        1.5       , 13.        ,  3.        ,  0.5       , 23.        ,
       11.5       ,  5.        ,  6.        , 10.        ,  7.        ,
        4.5       , 11.        , 14.        ,  7.5       , 17.        ,
        3.5       ,  7.16666667,  2.5       ,  6.5       , 10.5       ,
        3.16666667,  1.25      ,  8.        , 13.5       , 12.5       ,
       25.66666667,  8.5       ,  9.        , 11.66666667,  7.25      ,
        5.33333333,  5.58333333,  9.5       ])

In [22]:
apl = apl.sort_values(by='iris1')
apl["wpop"] = apl["accessibilite_weight"] * apl["pop_gp"]
apl["swpop"] = apl.groupby("iris1")["wpop"].transform("sum")
apl["R"] = apl["nb"] / (apl["swpop"] / 100000)
apl.head(5)
# apl["R"].unique()

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R
0,40080000,60530000,17,22,C,0.071361,167.589023,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.959365,118.600059,0.0
1,40080000,60990000,21,24,C,0.056135,1899.726451,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,106.640694,118.600059,0.0
2,40390000,61540000,24,30,C,0.027324,537.620953,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.689806,14.689806,0.0
3,40420000,60530000,11,19,C,0.102284,167.589023,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,144.237361,0.0
4,40420000,60710000,22,30,C,0.027324,677.528167,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.512592,144.237361,0.0


In [23]:
rgp = apl[apl["iris1"] == apl["iris2"]]
rgp = rgp[["iris1", "iris2", "type_iris", "pop_gp", "nb", "R", "swpop", "wpop", "pop_gp"]].copy()
rgp

,iris1,iris2,type_iris,pop_gp,nb,R,swpop,wpop,pop_gp
76,60010000,60010000,C,99.275089,0.0,0.000000,295.085798,99.275089,99.275089
87,60020000,60020000,C,59.473302,0.0,0.000000,254.968450,59.473302,59.473302
98,60030000,60030000,C,621.725124,0.0,0.000000,1230.503600,621.725124,621.725124
112,60040101,60040101,C,1725.263970,0.0,0.000000,190008.358917,1725.263970,1725.263970
477,60040102,60040102,A,2534.690523,2.0,1.317230,151833.796108,2534.690523,2534.690523
...,...,...,...,...,...,...,...,...,...
115797,61610105,61610105,A,3977.633694,5.0,2.438847,205014.939812,3977.633694,3977.633694
116147,61610106,61610106,A,3823.078668,0.0,0.000000,148752.497762,3823.078668,3823.078668
116501,61610107,61610107,A,2781.820360,0.0,0.000000,162704.435974,2781.820360,2781.820360
116506,61620000,61620000,C,848.449890,0.5,24.762201,2019.206588,848.449890,848.449890


In [24]:
apl = apl.merge(rgp, on="iris2", suffixes=("", "_dest"))
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest
0,40080000,60530000,17,22,C,0.071361,167.589023,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.959365,118.600059,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023
1,40420000,60530000,11,19,C,0.102284,167.589023,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,144.237361,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023
2,40760000,60530000,15,19,C,0.102284,167.589023,40760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,764.977605,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023
3,40900000,60530000,22,28,C,0.034735,167.589023,40900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,5.821248,57.728846,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023
4,41740000,60530000,14,20,C,0.090718,167.589023,41740000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,15.203333,150.770225,0.0,60530000,C,167.589023,0.0,0.0,396.976547,167.589023,167.589023


In [25]:
apl = apl.sort_values(by=['iris1', "iris2"])
apl["ap"] = apl["accessibilite_weight"] * apl["R_dest"]
apl["apl"] = apl.groupby("iris1")["ap"].transform("sum")
apl.head(5)

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,ap,apl
0,40080000,60530000,17,22,C,0.071361,167.589023,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.959365,118.600059,0.0,60530000,C,167.589023,0.0,0.000000,396.976547,167.589023,167.589023,0.000000,2.481017
12,40080000,60990000,21,24,C,0.056135,1899.726451,40080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,106.640694,118.600059,0.0,60990000,C,1899.726451,1.0,44.197516,2262.570591,1899.726451,1899.726451,2.481017,2.481017
33,40390000,61540000,24,30,C,0.027324,537.620953,40390000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,14.689806,14.689806,0.0,61540000,C,537.620953,1.0,102.836623,972.416215,537.620953,537.620953,2.809879,2.809879
1,40420000,60530000,11,19,C,0.102284,167.589023,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,17.141710,144.237361,0.0,60530000,C,167.589023,0.0,0.000000,396.976547,167.589023,167.589023,0.000000,4.740618
51,40420000,60710000,22,30,C,0.027324,677.528167,40420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,18.512592,144.237361,0.0,60710000,C,677.528167,1.0,82.697420,1209.227565,677.528167,677.528167,2.259601,4.740618


In [26]:
apl = apl[apl["iris1"] == apl["iris2"]]
apl

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,ap,apl
385,60010000,60010000,0,0,C,1.0,99.275089,60010000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,99.275089,295.085798,0.000000,60010000,C,99.275089,0.0,0.000000,295.085798,99.275089,99.275089,0.000000,32.968238
157,60020000,60020000,0,0,C,1.0,59.473302,60020000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,59.473302,254.968450,0.000000,60020000,C,59.473302,0.0,0.000000,254.968450,59.473302,59.473302,0.000000,0.000000
242,60030000,60030000,0,0,C,1.0,621.725124,60030000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,621.725124,1230.503600,0.000000,60030000,C,621.725124,0.0,0.000000,1230.503600,621.725124,621.725124,0.000000,16.998780
74748,60040101,60040101,0,0,C,1.0,1725.263970,60040101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1725.263970,190008.358917,0.000000,60040101,C,1725.263970,0.0,0.000000,190008.358917,1725.263970,1725.263970,0.000000,139.844501
74398,60040102,60040102,0,0,A,1.0,2534.690523,60040102,16425.0,DAEMERS,YVES,6.0,92989.0,7.072160,43.596934,16425_43.596934_7.07216,1.0,1.0,2.0,2534.690523,151833.796108,1.317230,60040102,A,2534.690523,2.0,1.317230,151833.796108,2534.690523,2534.690523,1.317230,111.858031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39136,61610105,61610105,0,0,A,1.0,3977.633694,61610105,99028.0,BENASSAYAG,JEAN JACQUES,6.0,68080.0,7.140329,43.639824,99028_43.639824_7.140329,1.0,1.0,5.0,3977.633694,205014.939812,2.438847,61610105,A,3977.633694,5.0,2.438847,205014.939812,3977.633694,3977.633694,2.438847,146.681286
38772,61610106,61610106,0,0,A,1.0,3823.078668,61610106,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3823.078668,148752.497762,0.000000,61610106,A,3823.078668,0.0,0.000000,148752.497762,3823.078668,3823.078668,0.000000,106.135482
38423,61610107,61610107,0,0,A,1.0,2781.820360,61610107,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2781.820360,162704.435974,0.000000,61610107,A,2781.820360,0.0,0.000000,162704.435974,2781.820360,2781.820360,0.000000,117.040350
116175,61620000,61620000,0,0,C,1.0,848.449890,61620000,16814.0,DI VINCENZO,DOMINIQUE,6.0,273951.0,7.612673,44.063665,16814_44.063665_7.612673,2.0,0.5,0.5,848.449890,2019.206588,24.762201,61620000,C,848.449890,0.5,24.762201,2019.206588,848.449890,848.449890,24.762201,59.947026


In [27]:
apl["apl"].describe() #mean=95 vs 98, median=101 vs 110

count    500.000000
mean      95.015498
std       50.531825
min        0.000000
25%       60.775840
50%      100.344054
75%      133.373725
max      488.109240
Name: apl, dtype: float64

In [28]:
apl[apl["apl"]>400]

,iris1,iris2,km,time,type_iris,accessibilite_weight,pop_gp,iris,id,nom,prenom,dept_id,adresse_norm_id,lon,lat,key,nb_cabinet,weight,nb,wpop,swpop,R,iris1_dest,type_iris_dest,pop_gp_dest,nb_dest,R_dest,swpop_dest,wpop_dest,pop_gp_dest,ap,apl
116464,61290000,61290000,0,0,C,1.0,317.755204,61290000,16295.0,PARODI CARIZZONI,CATHERINE,6.0,264607.0,7.103691,44.08307,16295_44.08307_7.103691,1.0,1.0,2.5,317.755204,546.040585,457.841426,61290000,C,317.755204,2.5,457.841426,546.040585,317.755204,317.755204,457.841426,488.10924


In [29]:
apl = apl[["iris1", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "wpop", "swpop", "R", "pop_gp"]]
apl = apl.rename(columns={'iris1': 'iris'})
apl.head(5)

,iris,type_iris,nb,apl,ap,accessibilite_weight,wpop,swpop,R,pop_gp
385,60010000,C,0.0,32.968238,0.00000,1.0,99.275089,295.085798,0.00000,99.275089
157,60020000,C,0.0,0.000000,0.00000,1.0,59.473302,254.968450,0.00000,59.473302
242,60030000,C,0.0,16.998780,0.00000,1.0,621.725124,1230.503600,0.00000,621.725124
74748,60040101,C,0.0,139.844501,0.00000,1.0,1725.263970,190008.358917,0.00000,1725.263970
74398,60040102,A,2.0,111.858031,1.31723,1.0,2534.690523,151833.796108,1.31723,2534.690523


In [30]:
def get_iriss():
    sql = f"""
select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 20{year} "year", {specialite} "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '{dept:02d}%'"""
    print(sql)
    return pd.read_sql(text(sql), config.connection_string)

iriss = get_iriss()
iriss


select i.id "iris", i.code "iris_string", i.nom "iris_label", c.dept_id "dept", c.code "code_commune", c.nom "commune_label", 2021 "year", 10 "specialite" from iris.iris i
join iris.commune c on c.id=i.commune_id
where i.code like '06%'


,iris,iris_string,iris_label,dept,code_commune,commune_label,year,specialite
0,60120105,060120105,Centre,6,06012,Beausoleil,2021,10
1,60500000,060500000,Coursegoules,6,06050,Coursegoules,2021,10
2,60720000,060720000,Ilonse,6,06072,Ilonse,2021,10
3,61050101,061050101,Nord,6,06105,Roquefort-les-Pins,2021,10
4,60150000,060150000,Berre-les-Alpes,6,06015,Berre-les-Alpes,2021,10
...,...,...,...,...,...,...,...,...
495,60883403,060883403,Madeleine Supérieure,6,06088,Nice,2021,10
496,60883501,060883501,Crémat-Bellet,6,06088,Nice,2021,10
497,60883601,060883601,Saint-Pierre de Féric,6,06088,Nice,2021,10
498,60883602,060883602,Costière,6,06088,Nice,2021,10


In [31]:
apl = apl.merge(iriss, on="iris", how="left", suffixes=("", "_dest"))
apl = apl[["year", "specialite", "iris", "iris_string", "type_iris", "nb", "apl", "ap", "accessibilite_weight", "R", "wpop", "swpop", "pop_gp", "iris_label", "dept", "code_commune", "commune_label"]]
apl

,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,iris_label,dept,code_commune,commune_label
0,2021,10,60010000,060010000,C,0.0,32.968238,0.000000,1.0,0.000000,99.275089,295.085798,99.275089,Aiglun,6,06001,Aiglun
1,2021,10,60020000,060020000,C,0.0,0.000000,0.000000,1.0,0.000000,59.473302,254.968450,59.473302,Amirat,6,06002,Amirat
2,2021,10,60030000,060030000,C,0.0,16.998780,0.000000,1.0,0.000000,621.725124,1230.503600,621.725124,Andon,6,06003,Andon
3,2021,10,60040101,060040101,C,0.0,139.844501,0.000000,1.0,0.000000,1725.263970,190008.358917,1725.263970,Les Trois Moulins,6,06004,Antibes
4,2021,10,60040102,060040102,A,2.0,111.858031,1.317230,1.0,1.317230,2534.690523,151833.796108,2534.690523,Les Semboules,6,06004,Antibes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2021,10,61610105,061610105,A,5.0,146.681286,2.438847,1.0,2.438847,3977.633694,205014.939812,3977.633694,Marina,6,06161,Villeneuve-Loubet
496,2021,10,61610106,061610106,A,0.0,106.135482,0.000000,1.0,0.000000,3823.078668,148752.497762,3823.078668,Vaugrenier,6,06161,Villeneuve-Loubet
497,2021,10,61610107,061610107,A,0.0,117.040350,0.000000,1.0,0.000000,2781.820360,162704.435974,2781.820360,Plan,6,06161,Villeneuve-Loubet
498,2021,10,61620000,061620000,C,0.5,59.947026,24.762201,1.0,24.762201,848.449890,2019.206588,848.449890,La Brigue,6,06162,La Brigue


In [32]:
# Anomalie = 1 seul medecin avec 1 seul cabinet pour 300 habitants contre 1 seul medecin et 0.33 cabinets
# La commune de sauveur en tinee a un APL > 400 pour les généralistes car l'iris a seulement 300 habitants jeunes et ils sont passés de 1 médecin avec 3 cabinets à 1 médecin a un seul cabinet en 2022
apl[apl["apl"] > 400]


,year,specialite,iris,iris_string,type_iris,nb,apl,ap,accessibilite_weight,R,wpop,swpop,pop_gp,iris_label,dept,code_commune,commune_label
436,2021,10,61290000,061290000,C,2.5,488.10924,457.841426,1.0,457.841426,317.755204,546.040585,317.755204,Saint-Sauveur-sur-Tinée,6,06129,Saint-Sauveur-sur-Tinée


In [34]:
apl.to_csv("apl06.csv", index=False)